In [3]:
import pandas as pd
import numpy as np
import requests, json
import os

In [5]:
client_id = os.environ['BNET_CLIENT_ID']
client_secret = os.environ['BNET_SECRET']

def create_access_token(client_id, client_secret, region = 'us'):
    data = { 'grant_type': 'client_credentials' }
    response = requests.post('https://%s.battle.net/oauth/token' % region, data=data, auth=(client_id, client_secret))
    return response.json()

token = create_access_token(client_id, client_secret)['access_token']

# Member List

In [6]:
members = requests.get(f'https://us.api.blizzard.com/data/wow/guild/barthilas/steamed-hams/roster?namespace=profile-us&locale=en_US&access_token={token}').json()

char_id = [i['character']['id'] for i in members['members']]
char_name = [i['character']['name'].lower() for i in members['members']]
char_level = [i['character']['level'] for i in members['members']]
char_class = [i['character']['playable_class']['id'] for i in members['members']]

members_df = pd.DataFrame(
    {'char_id':char_id,
     'char_name':char_name,
     'char_level':char_level,
     'char_class':char_class}
).set_index('char_name')
members_df = members_df[members_df['char_level'] == 70].sort_values('char_name')
members_df.head()


,char_id,char_level,char_class
char_name,,,
agnikai,203257707,70,9
ahmanva,181697502,70,9
antremodes,180625739,70,1
anxti,203254720,70,3
beardzdk,203791547,70,6


## Get current active members

In [7]:
time_list = []

for i in members_df.index:
    req = requests.get(f'https://us.api.blizzard.com/profile/wow/character/barthilas/{i}?namespace=profile-us&locale=en_US&access_token={token}').json()
    ts = req.get('last_login_timestamp')
    
    time_list.append(ts)
    
members_df['last_login'] = [i / 1000 for i in time_list]
members_df['last_login'] = pd.to_datetime(members_df['last_login'], unit='s')
members_df['date_diff'] = members_df.last_login - pd.Timestamp.now()

members_df['active_30_days'] = [i.days >= -30 for i in members_df.date_diff]
members_df['active_60_days'] = [i.days >= -60 for i in members_df.date_diff]
members_df

# Subset to currently active (30/60/etc. days)

In [ ]:
members_df = members_df[members_df['active_30_days']]
#members_df.to_csv('members_list.csv')
members_df

In [ ]:
# Add useful class, spec data